# Lab 03: Fun with `pandas`!

Below are some exercises to get you working with `pandas` to manipulate data. As always, get as far as you can, and ask for help when you need it! Your teacher (me), you instructor, and your classmates are all here to help each other get better at coding. Getting the code to work is important, but do also take the time to make sure you understand what the commands are doing. This time, (with the exception of the Stroop challenge), all I've given you is the code to download the data. Then you are on your own. For the Stroop challenge, I gave the you code for the first step—after that, it's up to you :-)

In [96]:
# Imports
import pandas as pd
from calendar import month_name
import pandas as pd
import os
import re

## Music sales challenge

Write a script that:

1. Combines the tables of best-selling physical singles and best-selling digital singles on the Wikipedia page "List_of_best-selling_singles"
2. Adds a column which marks whether each row is from the list of physical singles or digital singles
3. Outputs the artist and single name for the year you were born. If there is no entry for that year, take the closest year after you were born.
4. Outputs the artist and single name for the year you were 15 years old.

In [94]:
def load_and_combine_singles():
    url = "https://en.wikipedia.org/wiki/List_of_best-selling_singles"
    singles_list = pd.read_html(url)
    
    singles_data = [
        (singles_list[0], 'Physical'),
        (singles_list[1], 'Physical'),
        (singles_list[3], 'Digital'),
        (singles_list[4], 'Digital')
    ]
    
    combined_singles = pd.concat([
        df.assign(Type=single_type) for df, single_type in singles_data
    ], ignore_index=True)
    
    combined_singles['Released'] = pd.to_datetime(combined_singles['Released'], format='%Y', errors='coerce')
    return combined_singles.sort_values('Released')

def get_artist_and_single(df, year):
    entry = df[df['Released'].dt.year >= year].iloc[0]
    return f"{entry['Artist']} - {entry['Single']} ({entry['Released'].year})"

def main():
    combined_singles = load_and_combine_singles()
    # Save the combined singles data to a CSV file
    combined_singles.to_csv('combined_singles.csv', index=False)
    print(combined_singles.head(10))
    
    birth_year = 1995
    age_15_year = birth_year + 15
    
    print(f"\nArtist and single for my birth year: {get_artist_and_single(combined_singles, birth_year)}")
    print(f"\nArtist and single for when I was 15: {get_artist_and_single(combined_singles, age_15_year)}")

if __name__ == "__main__":
    main()

                     Artist                            Single   Released  \
2               Bing Crosby                    "Silent Night" 1935-01-01   
14      The Andrews Sisters           "Bei Mir Bist Du Schön" 1937-01-01   
8             The Ink Spots                "If I Didn't Care" 1939-01-01   
0               Bing Crosby                 "White Christmas" 1942-01-01   
31                Roy Acuff               "Wabash Cannonball" 1942-01-01   
28           Mills Brothers                      "Paper Doll" 1943-01-01   
3                Tino Rossi                 "Petit Papa Noël" 1946-01-01   
18               Gene Autry  "Rudolph the Red-Nosed Reindeer" 1949-01-01   
37               Patti Page                 "Tennessee Waltz" 1950-01-01   
4   Bill Haley & His Comets           "Rock Around the Clock" 1954-01-01   

       Sales (in millions)        Source      Type  
2                       30           [2]  Physical  
14                    14.0          [17]  Physical  
8   

## Space challenge

1. Make a single dataframe that combines the space missions from the 1950's to the 2020's
2. Write a script that returns the year with the most launches
3. Write a script that returns the most common month for launches
4. Write a script that ranks the months from most launches to fewest launches


In [87]:
def load_and_combine_space_missions():
    url = "https://en.wikipedia.org/wiki/Timeline_of_Solar_System_exploration"
    tables = pd.read_html(url)
    missions = pd.concat(tables[:8], ignore_index=True)
    missions['Launch date'] = pd.to_datetime(missions['Launch date'], format='%d %B %Y')
    print(f"Number of missions loaded: {len(missions['Launch date'])}\n")
    print(f"First five launch dates:\n{missions['Launch date'].head(5).to_string()})") # Verify we captured launch dates correctly
    return missions

def print_mission_counts(tables):
    print("\nLaunches by decade:")
    for i, table in enumerate(tables[:8], 1):
        decade = 1950 + (i-1) * 10
        print(f"{decade}'s: {len(table)}")

def analyze_launches(missions):
    launch_years = missions['Launch date'].dt.year
    launch_months = missions['Launch date'].dt.month

    year_with_most_launches = launch_years.value_counts().idxmax()
    most_common_month = launch_months.value_counts().idxmax()
    month_rankings = launch_months.value_counts().sort_values(ascending=False)
    
    return year_with_most_launches, most_common_month, month_rankings

def main():
    space_missions = load_and_combine_space_missions()
    
    space_missions_tables = pd.read_html("https://en.wikipedia.org/wiki/Timeline_of_Solar_System_exploration")
    print_mission_counts(space_missions_tables)

    year, month, rankings = analyze_launches(space_missions)
    print(f"\nYear with the most launches: {year}")
    print(f"Most common month for launches: {month_name[month]}")

    print("\nMonths ranked by number of launches (most to fewest):")
    rankings.index = rankings.index.map(lambda x: month_name[x])
    print(rankings.to_string())

    space_missions.to_csv('space_missions.csv', index=False)

if __name__ == "__main__":
    main()

Number of missions loaded: 233

First five launch dates:
0   1957-10-04
1   1957-11-03
2   1958-02-01
3   1958-03-17
4   1959-01-02)

Launches by decade:
1950's: 8
1960's: 73
1970's: 42
1980's: 14
1990's: 21
2000's: 24
2010's: 28
2020's: 23

Year with the most launches: 1965
Most common month for launches: November

Months ranked by number of launches (most to fewest):
Launch date
November     30
August       27
September    22
October      22
July         21
January      19
December     19
May          17
March        15
February     14
June         14
April        13


## Supervillain challenge

1. Write a script that combines the tables showing supervillain debuts from the 30's through the 2010's
2. Write a script that ranks each decade in terms of how many supervillains debuted in that decade
3. Write a script that ranks the different comics companies in terms of how many supervillains they have, and display the results in a nice table (pandas dataframe)

In [44]:
def extract_year(text):
    if pd.isna(text):
        return None
    match = re.search(r'\b(19\d{2}|20\d{2})\b', str(text))
    return int(match.group()) if match else None

def load_and_combine_supervillain_debuts():
    url = "https://en.wikipedia.org/wiki/List_of_comic_book_supervillain_debuts"
    tables = pd.read_html(url)

    os.makedirs("tables", exist_ok=True)

    # Initiate list of cleaned tables
    cleaned_tables = []

    for i, table in enumerate(tables[3:12]):
        # We check whether the set of columns exists, and extract only those (to avoid some of the hidden columns in the table structure)
        if set(table.columns) == set(['Character / Team', 'Year Debuted', 'Company', 'Creator/s', 'First Appearance']):
            cleaned_table = table[['Character / Team', 'Year Debuted', 'Company', 'Creator/s', 'First Appearance']]
            
            # Apply the year extraction to 'Year Debuted' column
            cleaned_table['Year Debuted'] = cleaned_table['Year Debuted'].apply(extract_year)
            
            # Log any rows where year extraction failed to catch errors
            failed_rows = cleaned_table[cleaned_table['Year Debuted'].isna()]
            if not failed_rows.empty:
                print(f"Failed to extract year for {len(failed_rows)} rows in table {i}:")
                print(failed_rows)
            
            # Append tables if cleaning is succesful. This ensures that the columns are identical across tables
            cleaned_tables.append(cleaned_table)
        
        # Save the files for inspection, to check that we correctly extracted the data from wikipedia.
        table.to_csv(f"tables/table_{1930+(i)*10}.csv", index=False)

    supervillains = pd.concat(cleaned_tables, ignore_index=True)
    
    return supervillains

def rank_decades(supervillains):
    # Floor division (divide by 10, then round down to nearest integer - this gets us decade), then multiply by 10 to get decade
    supervillains['Decade'] = (supervillains['Year Debuted'] // 10) * 10
    # Decade count after above calculation, then sort ascending
    debut_counts_by_decade = supervillains['Decade'].value_counts().sort_index(ascending=True).to_string() #  Add to_string() to avoid printing metadata
    return debut_counts_by_decade

def rank_comics_companies(supervillains):
    company_counts = supervillains['Company'].value_counts().sort_values(ascending=False).to_string()
    return company_counts

def main():
    supervillains = load_and_combine_supervillain_debuts()
    print(f"\nTotal number of supervillains: {len(supervillains)}")
    
    debut_counts_by_decade = rank_decades(supervillains)
    print("\nSupervillain debuts by decade:")
    print(debut_counts_by_decade)
    
    company_counts = rank_comics_companies(supervillains)
    print("\nComic companies ranked by number of supervillains:")
    print(company_counts)

    supervillains.to_csv('supervillains.csv', index=False)

if __name__ == "__main__":
    main()


Total number of supervillains: 636

Supervillain debuts by decade:
Decade
1930      4
1940     47
1950     26
1960    228
1970     97
1980     92
1990     84
2000     49
2010      9

Comic companies ranked by number of supervillains:
Company
DC                          338
Marvel                      264
Fawcett Comics/DC             6
Image                         5
Dark Horse                    5
Marvel/Timely                 4
Disney/Hyperion               4
Eternity                      3
Lev Gleason Publications      1
Comico                        1
Mirage                        1
Image Comics                  1


## Stroop challenge

Every year between 2015 and 2021, the students in my Language, Cognition, and the Brain course participated in a version of the Stroop task. Using a stopwatch (ok, using their phones), they recorded how fast they could say a list of things (either reading or naming colors or color words). The column names mean "Reading with No Interference", "Naming with Interference", "Naming with No Interference", and "Reading with Interference". The times are in seconds.

### Stroop challenge 1: 
Transform these data from wide format to long format, so that the result is a dataframe with
- 1 column named "Participant_id" with a unique number for each participant (you can use the row indices)
- 1 column named "Year" with the year data
- 1 column named "Task" that shows which task they were doing
- 1 column named "RT" that shows their response time

In [60]:
# Load the data
df = pd.read_csv("https://raw.githubusercontent.com/ethanweed/Stroop/master/Stroop-raw-over-the-years.csv")
df.columns = df.columns.str.lower()

# Reset index to create Participant_id
df = df.reset_index().rename(columns={'index': 'participant_id'})

df_long = pd.melt(df, 
                  id_vars=['participant_id', 'year'], 
                  value_vars=['reading_noint', 'naming_int', 'naming_noint', 'reading_int'],
                  var_name='task', 
                  value_name='rt')

print(df_long.head(5))

   participant_id  year           task    rt
0               0  2015  reading_noint  4.16
1               1  2015  reading_noint  4.35
2               2  2015  reading_noint  3.60
3               3  2015  reading_noint  3.90
4               4  2015  reading_noint  4.22


## Stroop challenge 2 (Advanced!!!):

Make a new dataframe which shows the mean response time (in seconds) for each task for each year.

In [95]:
# Create a new dataframe with mean response times
df_mean = df_long.groupby(['year', 'task'])['rt'].mean().reset_index()

# Pivot the table to have tasks as columns
df_mean_wide = df_mean.pivot(index='year', columns='task', values='rt')
print(df_mean_wide.head(5))

task  naming_int  naming_noint  reading_int  reading_noint
year                                                      
2015    8.617143      5.123571     4.446429       3.951429
2016    8.859268      5.405610     5.340000       4.076098
2017    9.311765      5.771176     5.492353       4.414412
2018    9.372667      5.298000     4.938667       3.886000
2019    9.536087      6.345652     6.090870       4.935652
